In [1]:
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
from implementations import * 

In [2]:
data_path = "data/train.csv"

In [3]:
y, x, ids = load_csv_data(data_path, sub_sample=False)

In [4]:
tx, mean_x, std_x = standardize(x)

In [5]:
#SGD parameters
initial_w = np.zeros((x.shape[1]))
max_iters = 100
gamma = 0.05
lambda_ = 0.7

In [6]:
w_ls_sg, loss_ls_gd = least_squares_GD(y, tx, initial_w, max_iters, gamma)

In [7]:
w_ls_sg

array([ 0.2035595 , -0.12962203, -0.05527349,  0.01229099, -0.02450509,
        0.19385846, -0.02780161, -0.02976163, -0.0456469 , -0.03147113,
       -0.03162885, -0.02774737, -0.02520986,  0.00993751, -0.02963122,
       -0.02967984, -0.05291429, -0.02962913, -0.02952568, -0.0311228 ,
       -0.02949501, -0.05152506, -0.03020389,  0.02824165,  0.01292121,
        0.01291545, -0.03789906, -0.02545391, -0.02550089, -0.04770133])

In [8]:
w_ls_sgd, loss_ls_sgd = least_squares_SGD(y, tx, initial_w, max_iters, gamma)

In [9]:
w_ls, loss_ls = least_squares(y, tx)

In [10]:
w_rr, loss_rr = ridge_regression(y, tx, lambda_)

In [11]:
w_lr, loss_lr = logistic_regression(y, tx, initial_w, max_iters, gamma)

In [12]:
w_rlr, loss_rlr = reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma)

In [14]:
%time w_cv, loss_cv = perform_cross_validation(y, tx, compute_loss_least_squares, compute_gradient_least_squares , 100)

CPU times: user 31min 34s, sys: 51.4 s, total: 32min 25s
Wall time: 5min 26s


In [15]:
w_cv, loss_cv

(array([ 0.58958322, -0.04841343,  0.18137234,  0.17040102, -0.42563101,
        -0.35543422, -0.43606014,  0.06936882,  0.01374764,  0.37435468,
         0.06904974,  0.06896532, -0.42796274,  0.24996587,  0.06368095,
         0.06637236,  0.15898876,  0.06241744,  0.07550853,  0.07629161,
         0.07689515,  0.22913112,  0.07030294, -0.3000637 , -0.41003288,
        -0.40870228, -0.45978196, -0.43191981, -0.42945674,  0.10787678]),
 10.05194093893734)